## 네이버 지식in

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import numpy as np
import pickle
import re

In [2]:
data = pd.read_csv('./data/민사변호사_프로필_링크.csv', encoding='utf-8')
data

,변호사이름,프로필링크
0,서동민,https://kin.naver.com//profile/index.naver?u=a...
1,김광웅,https://kin.naver.com//profile/index.naver?u=o...
2,권광중,https://kin.naver.com//profile/index.naver?u=6...
3,이재호,https://kin.naver.com//profile/index.naver?u=7...
4,박준상,https://kin.naver.com//profile/index.naver?u=c...
...,...,...
72,박진세,https://kin.naver.com//profile/index.naver?u=d...
73,권희영,https://kin.naver.com//profile/index.naver?u=e...
74,안지성,https://kin.naver.com//profile/index.naver?u=i...
75,김도훈,https://kin.naver.com//profile/index.naver?u=D...


In [3]:
browser = Chrome()

In [65]:
detail_btn = browser.find_element(By.CSS_SELECTOR, '#container > div > div > div.container-fluid-content__left > div > div > ul > li:nth-child(1) > ul > li > a')

In [67]:
ori_url = browser.current_url
ori_url

'https://kin.naver.com/userinfo/expert/answerList.naver?u=az9GaUwMSdvq1LHyaTTg0tL7QX%2FHlOX3TUKF3tAy2xg%3D'

In [10]:
url_lst = []
for url in data['프로필링크'][59:]:
    browser.get(url)
    time.sleep(1)
    detail_btn = browser.find_element(By.CSS_SELECTOR, '#container > div > div > div.container-fluid-content__left > div > div > ul > li:nth-child(1) > ul > li > a')
    detail_btn.click()
    time.sleep(1)
    
    ori_url = browser.current_url
    qna_num = int(browser.find_element(By.CSS_SELECTOR, '#content > dl > dd:nth-child(2)').text)
    page_num = qna_num // 20
    
    # first page
    for n in range(1, 21):
        cat = browser.find_element(By.CSS_SELECTOR, f'#au_board_list > tr:nth-child({n}) > td.field > a').text
        if cat in ['민사소송','민사집행']:
            temp = browser.find_element(By.CSS_SELECTOR, f'#au_board_list > tr:nth-child({n}) > td.title > a').get_attribute('href')
            url_lst.append(temp)
    
    # Other pages
    for i in range(2, page_num+1):
        next_url = ori_url + '&page=' + str(i)
        browser.get(next_url)
        time.sleep(1)
        try:
            for n in range(1, 21):
                cat = browser.find_element(By.CSS_SELECTOR, f'#au_board_list > tr:nth-child({n}) > td.field > a').text
                if cat in ['민사소송','민사집행']:
                    temp = browser.find_element(By.CSS_SELECTOR, f'#au_board_list > tr:nth-child({n}) > td.title > a').get_attribute('href')
                    url_lst.append(temp)
        except:
            browser.refresh()
            pass

In [21]:
len(url_lst)

1651

In [12]:
import pickle
with open('url_lst_3.pickle', 'wb') as f:
    pickle.dump(url_lst, f)

In [20]:
with open('url_lst_3.pickle', 'rb') as f:
    url_lst = pickle.load(f)

In [22]:
browser = Chrome()

titles = []
bodys = []
replies = []

for url in tqdm(url_lst):
    try:
        browser.get(url)
        time.sleep(1)
        
        title = browser.find_element(By.CSS_SELECTOR, '#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__title > div > div').text
        body = browser.find_element(By.CSS_SELECTOR, '#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__content').text
        reply = browser.find_element(By.CSS_SELECTOR, '#answer_1 > div._endContents.c-heading-answer__content > div._endContentsText.c-heading-answer__content-user').text

        titles.append(title)
        bodys.append(body)
        replies.append(reply)
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████| 1651/1651 [46:41<00:00,  1.70s/it]


In [23]:
data = pd.DataFrame({'제목':titles, '본문':bodys, '답변':replies})
print(data.shape)
data

(1405, 3)


,제목,본문,답변
0,명도소송 문의드립니다,안녕하세요\n저희 서울집의 반지하 사람이 쓰레기와 오물 집기 이런것들을 내려가는 계...,안녕하세요. 로시컴-네이버 지식iN 상담변호사 김기윤 입니다.\n임차인이 임대차 목...
1,소액거래? 사기,밑에 요약 있어요\n\n안녕하세요. 다름이 아니고 소액거래? 사기에 관해서 문제가 ...,안녕하세요. 로시컴-네이버 지식iN 상담변호사 김기윤 입니다.\n사기죄는 타인을 기...
2,민사소송하는법,휴대폰 사기로 고소를 진행 했었습니다.\n2020/2/4 날짜로 법원에서 판결문이 ...,안녕하세요. 로시컴-네이버 지식iN 상담변호사 김기윤 입니다.\n1.지금까지 상대방...
3,대리해주실 변호사님을 모십니다.,"1) 항소심에 도움주실 변호사님을 원합니다.\n2) 액수는 3,000,000원으로 ...","안녕하세요. 로시컴-네이버 지식iN 상담변호사 김기윤 입니다.\n송대리, 소송진행,..."
4,외국인에게 돈을 빌려주고 못 받고 잇습니다,같은회사 외국인 노동자에게\n\n6월 말에 소액을 빌려주고 7월 초에 돈을 좀더 크...,안녕하세요. 로시컴-네이버 지식iN 상담변호사 김기윤 입니다.\nA. 외국인에게도 ...
...,...,...,...
1400,폭행 민사 고소,안녕하세요 \n폭행 민사 고소에 대하여 여쭤보고 싶어서 질문드립니다.\n이미 형사 ...,안녕하세요. 네이버법률 소속 - 네이버 지식iN 상담 변호사 조기현 입니다.\n질문...
1401,이런경우도 명예회손에 해당되나요?,상가 주인이 가게를 빼달라고 해서 가게를 정리 하던 중에 보증금을 받은적이 없는데 ...,안녕하세요. 네이버법률 소속 - 네이버 지식iN 상담 변호사 조기현 입니다.\n질문...
1402,이게 명예훼손 고소 꺼리 될까요?,익명의 답변자가 답변으 너무 성의없이\n\n인터넷에 치면 나와 라고 적었길래\n댓글...,안녕하세요. 서울지방변호사회-네이버 지식iN 상담변호사 조기현 입니다.\n▶ 명예훼...
1403,친정오빠 사망후,지금 친정아빠와 저 둘이 남아있는 상태입니다\n다름아니오라 친정오빠가 갑자기 사망을...,안녕하세요. 로시컴-네이버 지식iN 상담변호사 김연준 입니다.\n1순위 상속인이이 ...


In [24]:
data.drop_duplicates(subset=['본문'], inplace=True)
data.to_csv('Legal_QnA(41~59).csv', index=False, encoding='utf-8')

In [25]:
data_1 = pd.read_csv('./data/Naver_QnA(~41).csv', encoding='utf-8')
data_2 = pd.read_csv('./data/Naver_QnA(41~59).csv', encoding='utf-8')
data_3 = pd.read_csv('./data/Naver_QnA(59~).csv', encoding='utf-8')

In [30]:
f_data = pd.concat([data_1, data_2, data_3])
f_data

,제목,본문,답변
0,"민사 관련) 차량은 제 명의지만 아버지가 사용, 집은 어머니 명의",안녕하세요...\n정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 부디 자...,안녕하세요. 로시컴-네이버 지식iN 상담변호사 서동민 입니다.\n차량의 경우 현재 ...
1,불기소이유통지 10년뒤 부당이득금 기소,10년전에 대포통장이로 인해 피해을 봐서 재판결과가 불기소이유통지을 받았거든요\n근...,안녕하세요. 로시컴-네이버 지식iN 상담변호사 서동민 입니다.\n소장 내용에 대한 ...
2,"공증받은 차용증, 민사 청구이의소 관련해서 질문이 있습니다","연고도 다르고, 일면식도 없는 5살위의 형을, 일을 통해 알게 됐습니다.\n제가 주...",안녕하세요. 로시컴-네이버 지식iN 상담변호사 서동민 입니다.\n별도의 구체적 상담...
3,청구이의소송? 채무부존재확인소송 가능할까요?,1) 2010.06.15 이행권고결정(판결)\n2) 2011.05.23 채권압류...,안녕하세요. 로시컴-네이버 지식iN 상담변호사 서동민 입니다.\n별도의 구체적 상담...
4,사기를 당했는데 피의자가 구치소에 수감이 됬습니다 어떻게 받아내야 하나요ㅠ,4천만원정도의 피해가 있고 고소를 했는데 인지사건으로 넘어가서 검찰에서 수사중이었습...,안녕하세요. 로시컴-네이버 지식iN 상담변호사 서동민 입니다.\n가해자가 위 사건으...
...,...,...,...
1896,상간녀소송 질문이요!,재혼한지 5년정도되었어요\n근데 지금 남편이 전부인집에 들락날락 거리는것같아요\n전...,안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 이도형 입니다.\n간통죄가...
1897,"인테리어 시공사 무단침입, 무단촬영 법적조치",인테리어 시공사가 공사완료된 후 집주인 동의없이 무단으로 지인들과 집에 들어가서 사...,안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 이도형 입니다.인테리어 시...
1898,협박죄로 민사소송을 하려하는데 공소시효가 몇 년인가요?,"상대방이 협박, 모욕으로 약식명령 벌금 100만원이 나왔습니다.\n\n민사소송을 제...",안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 이도형 입니다.\n상대방의...
1899,이런 경우 민사소송으로 해결가능한가요?,중고거래를 했는데 판매자 분이 상자에 책을 담으면서 그 어떤 완충재도 없이 책만 달...,안녕하세요. 서울지방변호사회-네이버 지식iN 상담 변호사 이도형 입니다.\n배송 중...


In [31]:
f_data.isna().sum()

제목    0
본문    0
답변    0
dtype: int64